In [516]:
import numpy as np
import pandas as pd
import re
import words2num as w2n

In [517]:
Data = pd.read_excel(r"D:\VŠE\13_4IT500_Datový projekt\Data\odpovedi_202105081142.xlsx")

In [518]:
Data

,ID,DOTAZNIK,OTAZKA,PEDAGOG,TYP_AKCE,HODNOTA_CISLO,HODNOTA_CLOB
0,199,364350,21,NaN,NaN,NaN,12
1,207,364350,6,NaN,NaN,3.0,NaN
2,208,364350,22,NaN,NaN,4.0,NaN
3,924,313588,27,65042.0,NaN,NaN,NaN
4,925,313588,23,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
105405,1890717,495513,92,NaN,NaN,NaN,3
105406,1890718,495513,87,51194.0,NaN,4.0,NaN
105407,1890719,495513,85,57039.0,NaN,4.0,NaN
105408,1890720,495513,89,NaN,NaN,4.0,NaN


In [519]:
# Selekce otázek, které spadají pod otázku - Počet hodin strávených týdně
data_hodiny = Data.loc[Data["OTAZKA"].isin([21,42,92])].copy()

In [520]:
# Převedení na malá písmena
data_hodiny["HODNOTA_CLOB"] = data_hodiny['HODNOTA_CLOB'].str.lower()

In [521]:
data_hodiny.head(30)

,ID,DOTAZNIK,OTAZKA,PEDAGOG,TYP_AKCE,HODNOTA_CISLO,HODNOTA_CLOB
0,199,364350,21,NaN,NaN,NaN,12
11,932,313588,21,NaN,NaN,NaN,nula
46,2343,313591,21,NaN,NaN,NaN,jedna
51,1587,325364,21,NaN,NaN,NaN,nula
64,3607,313589,21,NaN,NaN,NaN,6
66,3328,325363,21,NaN,NaN,NaN,2
73,3091,318948,21,NaN,NaN,NaN,2-3 hodiny
102,3405,324745,21,NaN,NaN,NaN,14 hodin
105,3575,328492,21,NaN,NaN,NaN,mezi 0 a 20 hodinama
140,4349,326557,21,NaN,NaN,NaN,3


In [523]:
# Definování slovníku - pro transformaci textových hodnot do číselných (potenciál pro zlepšení ve skloňování)
dicCZ = {"nula":"0","jedna":"1","dvě":"2","tři":"3","čtyři":"4","pět":"5","šest":"6","sedm":"7","osm":"8","devět":"9","deset":"10"}

In [524]:
# Definování RegEx funkcí

def hodin(imp):
    try:
        x = re.sub("h[\w]*","",imp)
        return x
    except TypeError:
        return imp

def tecka(imp):
    try:
        x = re.sub(",",".",imp)
        return x
    except TypeError:
        return imp

def delimeter(imp):
    try:
        x = re.sub('-',"",imp)
        return x
    except TypeError:
        return imp

def get_list_digits(imp):
    try:
        x = re.findall('[0-9][.,][0-9]|\d+',imp)
        #x = re.findall("^\d*[.,]?\s?\d*$",NumList)
        return x
    except TypeError:
        return imp
    
def Word2numEng(inp):
    try:
        value = w2n.words2num(inp)
        return value
    
    except ValueError:
        return inp
    
    except AttributeError:
        return inp
    
def get_list_float(x):
    try:
        return list(map(float, x))
    except ValueError:
        pass

In [525]:
# Aplikování funkcí a převedení do stringu
data_hodiny.replace(to_replace = dicCZ,inplace = True)
data_hodiny["HODNOTA_CLOB"] = data_hodiny["HODNOTA_CLOB"].apply(hodin)
data_hodiny["HODNOTA_CLOB"] = data_hodiny["HODNOTA_CLOB"].apply(delimeter)
data_hodiny["HODNOTA_CLOB"] = data_hodiny["HODNOTA_CLOB"].apply(tecka)
data_hodiny["HODNOTA_CLOB"] = data_hodiny["HODNOTA_CLOB"].apply(lambda x: Word2numEng(x))
data_hodiny["HODNOTA_CLOB"] = data_hodiny["HODNOTA_CLOB"].astype(str)

In [526]:
# Funkce na získání číselných hodnot a převedení na float
data_hodiny["HODNOTA_CLOB_LIST"] = data_hodiny["HODNOTA_CLOB"].apply(get_list_digits)
data_hodiny["HODNOTA_CLOB_FLOAT"] = data_hodiny["HODNOTA_CLOB_LIST"].apply(get_list_float)

In [547]:
data_hodiny.head(30)

,ID,DOTAZNIK,OTAZKA,PEDAGOG,TYP_AKCE,HODNOTA_CISLO,HODNOTA_CLOB,HODNOTA_CLOB_LIST,HODNOTA_CLOB_FLOAT,HODNOTA_CLOB_AVG
0,199,364350,21,NaN,NaN,NaN,12,[12],[12.0],12.0
11,932,313588,21,NaN,NaN,NaN,0,[0],[0.0],0.0
46,2343,313591,21,NaN,NaN,NaN,1,[1],[1.0],1.0
51,1587,325364,21,NaN,NaN,NaN,0,[0],[0.0],0.0
64,3607,313589,21,NaN,NaN,NaN,6,[6],[6.0],6.0
66,3328,325363,21,NaN,NaN,NaN,2,[2],[2.0],2.0
73,3091,318948,21,NaN,NaN,NaN,23,[23],[23.0],23.0
102,3405,324745,21,NaN,NaN,NaN,14,[14],[14.0],14.0
105,3575,328492,21,NaN,NaN,NaN,mezi 0 a 20,"[0, 20]","[0.0, 20.0]",10.0
140,4349,326557,21,NaN,NaN,NaN,3,[3],[3.0],3.0


In [534]:
# Výpočet průměru z listu hodnot
data_hodiny["HODNOTA_CLOB_AVG"] = data_hodiny["HODNOTA_CLOB_FLOAT"].apply(np.mean)

D:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [546]:
data_hodiny.describe()

,ID,DOTAZNIK,OTAZKA,PEDAGOG,TYP_AKCE,HODNOTA_CISLO,HODNOTA_CLOB_AVG
count,6.037000e+03,6037.000000,6037.000000,0.0,0.0,0.0,5.742000e+03
mean,9.961030e+05,412749.257413,58.070399,NaN,NaN,NaN,2.846445e+04
std,5.587539e+05,65486.988965,31.749285,NaN,NaN,NaN,2.154903e+06
min,1.990000e+02,290088.000000,21.000000,NaN,NaN,NaN,0.000000e+00
25%,4.847710e+05,337828.000000,21.000000,NaN,NaN,NaN,1.000000e+00
50%,1.062713e+06,419717.000000,42.000000,NaN,NaN,NaN,2.000000e+00
75%,1.476474e+06,472731.000000,92.000000,NaN,NaN,NaN,4.000000e+00
max,1.890717e+06,510576.000000,92.000000,NaN,NaN,NaN,1.632899e+08


In [551]:
# Uložení dat
data_hodiny.to_excel(r'D:\VŠE\13_4IT500_Datový projekt\Power Bi - MVP\data\odpovedi_hodiny.xlsx', index = False)

# Pouze ke kontrole během psaní

In [552]:
data_hodiny["HODNOTA_CLOB_FLOAT"] = data_hodiny.iloc[0:30,7].apply(lister)

In [537]:
data_hodiny.dtypes

ID                      int64
DOTAZNIK                int64
OTAZKA                  int64
PEDAGOG               float64
TYP_AKCE              float64
HODNOTA_CISLO         float64
HODNOTA_CLOB           object
HODNOTA_CLOB_LIST      object
HODNOTA_CLOB_FLOAT     object
HODNOTA_CLOB_AVG      float64
dtype: object

In [530]:
data_hodiny.loc[data_hodiny["OTAZKA"]==92]
# otazky 42, 42, 92

,ID,DOTAZNIK,OTAZKA,PEDAGOG,TYP_AKCE,HODNOTA_CISLO,HODNOTA_CLOB,HODNOTA_CLOB_LIST,HODNOTA_CLOB_FLOAT
54090,1150807,457713,92,NaN,NaN,NaN,1,[1],[1.0]
54125,1154946,472658,92,NaN,NaN,NaN,3,[3],[3.0]
54187,1146938,449258,92,NaN,NaN,NaN,0,[0],[0.0]
54203,1147217,454706,92,NaN,NaN,NaN,2,[2],[2.0]
54270,1147872,482380,92,NaN,NaN,NaN,více než 15,[15],[15.0]
...,...,...,...,...,...,...,...,...,...
105321,1890498,510453,92,NaN,NaN,NaN,4,[4],[4.0]
105339,1890443,454826,92,NaN,NaN,NaN,3,[3],[3.0]
105364,1890676,482507,92,NaN,NaN,NaN,5,[5],[5.0]
105396,1890589,478030,92,NaN,NaN,NaN,5,[5],[5.0]


In [543]:
Data.loc[Data["HODNOTA_CLOB"] == "nevim" ]

,ID,DOTAZNIK,OTAZKA,PEDAGOG,TYP_AKCE,HODNOTA_CISLO,HODNOTA_CLOB
61221,1240035,457720,97,NaN,NaN,NaN,nevim
85412,1532034,451451,83,NaN,NaN,NaN,nevim
88510,1532019,459711,90,NaN,NaN,NaN,nevim
94213,1785610,501135,83,NaN,NaN,NaN,nevim


In [550]:
data_hodiny.loc[data_hodiny["HODNOTA_CLOB"] == 'nevim']

,ID,DOTAZNIK,OTAZKA,PEDAGOG,TYP_AKCE,HODNOTA_CISLO,HODNOTA_CLOB,HODNOTA_CLOB_LIST,HODNOTA_CLOB_FLOAT,HODNOTA_CLOB_AVG
